In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime
import threading

In [3]:
jumlah_index = 1
threads_link = []
links = []
results = []
threads = []

In [4]:
def scrape_links(page_number):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
    }
    url = f"https://radarsemarang.jawapos.com/jateng?page={page_number}"
    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    articles = soup.find_all('h2',{"class": "latest__title"})
    
    page_links = []
    for article in articles:
        link = article.find('a')['href']
        page_links.append(link)
    
    print(f"Scraped {len(page_links)} links from page {page_number}")
    return page_links

In [5]:
for page_number in range(1, jumlah_index + 1):
    thread = threading.Thread(target=lambda p=page_number: links.extend(scrape_links(p)))
    thread.start()
    threads_link.append(thread)

for thread in threads_link:
    thread.join()
print("Total Links:", len(links))

Scraped 20 links from page 1
Total Links: 20


In [6]:
def scrape_url(url):
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'
        }
        response = requests.get(url,headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')        
            # Judul Berita
            title_elem = soup.find('h1',{"class": "read__title"})
            if title_elem:
                title_text = title_elem.text
            else:
                title_text = "Title not found"
            # Author berita
            author_elem = soup.find('div', {"class": "read__info__author"})
            if author_elem:
                author_text = author_elem.find('a')
                author_text = author_text.text
            else:
                author_text = "Author not found"     
            # tanggal berita
            date_elem = soup.find('div', {"class": "read__info__date"})
            if date_elem:
                date_text = date_elem.text
            else:
                date_text = "Date not found"
            #     # Category berita
            category_elements = soup.find_all('a', {"class": "breadcrumb__link"})
            if category_elements:
                category_text= category_elements[1].text
            else:
                category_text = "Category not found"
            #     # Content Berita
            body_elem = soup.find('article', {"class": "read__content clearfix"})
            
            if body_elem:
                content_elem = body_elem.find_all('div')
                content_text = ""
                for p in content_elem:
                    content_text += p.text.strip() + "\n"
                
                if content_text.strip():
                    content_text=content_text
                else:
                    content_text ="Content not found"
            else:
                content_text ="Content not found"

            results.append({'title': title_text,
                            'author' : author_text,
                            'category':category_text,
                            'date': date_text,
                            'content' : content_text,
                            'region':'jateng',
                            'link' : url})
        else:
            print(f"Failed to retrieve data from {url}")

    except requests.exceptions.RequestException as e:
        print(f"Error fetching URL '{url}': {e}")
    except Exception as e:
        print(f"Error processing URL '{url}': {e}")

In [7]:
for url in links:
    thread = threading.Thread(target=scrape_url, args=(url,))
    thread.start()
    threads.append(thread)
    
for thread in threads:
    thread.join()

Failed to retrieve data from https://radarsemarang.jawapos.com/jateng/723012596/ferry-wawan-cahyono-dorong-percepatan-penanganan-lonjakan-kenaikan-harga-beras-di-jateng


In [8]:
df = pd.DataFrame(results)
print(len(results))
df.head(5)

19


,title,author,category,date,content,region,link
0,"Resmi Dilantik Jadi Pj Bupati Temanggung, Har...",Khafifah Arini Putri,Jateng,\n - Minggu...,Content not found,jateng,https://radarsemarang.jawapos.com/jateng/72301...
1,"Operasi Sikat Jaran Candi 2023, Polda Jateng ...",Muhammad Hariyanto,Jateng,\n - Minggu...,Content not found,jateng,https://radarsemarang.jawapos.com/jateng/72297...
2,Pelaku Pembunuhan Gadis Berseragam Pramuka Se...,Lutfi Hanafi,Pemalang,"\n - Senin,...",Content not found,jateng,https://radarsemarang.jawapos.com/pemalang/723...
3,"714 Desa di Jateng Krisis Air Bersih, Blora d...",Khafifah Arini Putri,Jateng,"\n - Rabu, ...",Content not found,jateng,https://radarsemarang.jawapos.com/jateng/72295...
4,Akan Dibuka BRT Trans Jateng Koridor Baru Pan...,Miftahul A’la,Jateng,"\n - Senin,...",Content not found,jateng,https://radarsemarang.jawapos.com/jateng/72297...


In [9]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
excel_file_name = f'../../tempat_hasil_daerah/radar_semarang_{current_datetime}.xlsx'
df.to_excel(excel_file_name, index=False)

print(f'Data has been saved to {excel_file_name}')